In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [3]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [4]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [5]:
def my_loss(real_data, gen_data):
    err = real_data - gen_data
    mse = tf.math.reduce_mean(tf.math.square(err))
    y_pred = dnn_model(gen_data)
    b = tf.math.divide(y_pred, 100000)
    b = tf.math.square(b)

    loss = tf.math.add(tf.math.multiply(mse, 2), b)
    return loss

In [6]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='tanh'))

generator.compile(loss=my_loss, optimizer='adam')

In [7]:
def return_latest():
    df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    return X[-1]

In [8]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    if i % 10 == 0:
        print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[2.3586683]], pop: [[37.589573]]
epoch: 10, loss: [[2.1327908]], pop: [[41.8522]]
epoch: 20, loss: [[1.5575552]], pop: [[39.66579]]
epoch: 30, loss: [[2.2169325]], pop: [[40.7542]]
epoch: 40, loss: [[3.297326]], pop: [[42.062237]]
epoch: 50, loss: [[2.32229]], pop: [[40.647778]]
epoch: 60, loss: [[2.5195203]], pop: [[40.38322]]
epoch: 70, loss: [[1.9757067]], pop: [[39.886715]]
epoch: 80, loss: [[1.7739415]], pop: [[39.487293]]
epoch: 90, loss: [[3.0806456]], pop: [[40.90259]]
epoch: 100, loss: [[2.3799086]], pop: [[39.821804]]
epoch: 110, loss: [[1.8867396]], pop: [[40.94074]]
epoch: 120, loss: [[3.378153]], pop: [[41.02762]]
epoch: 130, loss: [[1.1879398]], pop: [[40.81262]]
epoch: 140, loss: [[2.6613853]], pop: [[41.45104]]
epoch: 150, loss: [[1.7816528]], pop: [[40.45324]]
epoch: 160, loss: [[1.3731868]], pop: [[40.978916]]
epoch: 170, loss: [[0.8101455]], pop: [[38.950695]]
epoch: 180, loss: [[1.4712676]], pop: [[40.16362]]
epoch: 190, loss: [[3.0483434]], pop: [[

In [10]:
result = np.array(result).reshape(1000, 21)
pop_result = np.array(pop_result).reshape(1000, 1)
X_result = scaler.inverse_transform(result)

pd_result = pd.DataFrame(X_result)
pd_result.to_excel('basic_formula.xlsx')

pd_pop_result = pd.DataFrame(pop_result)
pd_pop_result.to_excel('basic_formula_pop.xlsx')